# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # Implement Function
    wordList = []
    for line in text:
        lineWords = line.split()
        wordList.extend(lineWords)
    words = tuple(set(wordList))
    int2word = dict(enumerate(words))
    word2int = {word: ii for ii, word in int2word.items()}
    # return tuple
    return word2int, int2word

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # Implement Function
    dictionary = {
        ".": "||Period||",
        ",": "||Comma||",
        "\"": "||Quatation_Mark||",
        ";": "||Semicolon||",
        "!": "||Exclamation_Mark||",
        "?": "||Question_Mark||",
        "(": "||Left_Parentheses",
        ")": "||Right_Parentheses||",
        "-": "||Dash||",
        "\n": "||Return||"
    };
    
    return dictionary

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [8]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    features = []
    targets = []
    # iterate through the array, one sequence at a time
    for n, val in enumerate(words):
        # The features
        try:
            feature = words[n:n+sequence_length]
            target = words[n+sequence_length]
        except IndexError:
            continue
        
        features.append(feature)
        targets.append(target)
    # return a dataloader
    feature_tensor = torch.tensor(features)
    target_tensor = torch.tensor(targets)
    data = TensorDataset(feature_tensor, target_tensor)
    data_loader = torch.utils.data.DataLoader(data, 
                                                batch_size=batch_size,
                                                shuffle=True)
    
    return data_loader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [9]:
# test dataloader

test_text = list(range(50))
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[18, 19, 20, 21, 22],
        [30, 31, 32, 33, 34],
        [ 6,  7,  8,  9, 10],
        [29, 30, 31, 32, 33],
        [12, 13, 14, 15, 16],
        [13, 14, 15, 16, 17],
        [26, 27, 28, 29, 30],
        [ 3,  4,  5,  6,  7],
        [41, 42, 43, 44, 45],
        [ 9, 10, 11, 12, 13]])

torch.Size([10])
tensor([23, 35, 11, 34, 17, 18, 31,  8, 46, 14])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [10]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # Implement function
        
        # set class variables
        self.vocab_size = vocab_size
        self.output_size = output_size,
        self.n_embedding = embedding_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.dropout = dropout
        
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        
        # define model layers
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=dropout, batch_first=True)
        #self.lstm2 = nn.LSTM(hidden_dim, hidden_dim, n_layers, 
        #                    dropout=dropout, batch_first=True)
        
        self.dropout = nn.Dropout(dropout)
        
        self.fc = nn.Linear(hidden_dim, output_size)
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        batch_size = nn_input.size(0)
        # Implement function   
        embed_output = self.embed(nn_input)
        
        lstm_output, hidden = self.lstm(embed_output, hidden)
        #r_output, hidden = self.lstm2(r_output, hidden)
        ## pass through a dropout layer
        lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)

        # dropout layer after LSTM
        #output = self.dropout(lstm_output)
        output = lstm_output#let lstm handle dropout?
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        #out = out.contiguous().view(-1, self.n_hidden)
        #out = out.view(forward_batch_size, -1, self.output_size)

        output = self.fc(output)
        ## put x through the fully-connected layer
        output_size = self.output_size[0]
        output = output.view(batch_size, -1, output_size)
        output = output[:, -1]
        # return the final output and the hidden state
        return output, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        
        # initialize hidden state with zero weights, and move to GPU if available
        weight = next(self.parameters()).data
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [11]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # Implement Function
    # move data to GPU, if available
    if(train_on_gpu):
        inp, target = inp.cuda(), target.cuda()
        
    hidden = tuple([each.data for each in hidden])

    rnn.zero_grad()
    # outputs from the rnn
    prediction, hidden = rnn(inp, hidden)
    # perform backpropagation and optimization
    # calculate the loss
    loss = criterion(prediction, target)

    # perform backprop and update weights
    loss.backward()
    nn.utils.clip_grad_norm_(rnn.parameters(), 5)
    optimizer.step()
    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import datetime

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('{} - Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    datetime.datetime.now(), epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of unique tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [13]:
# Data params
# Sequence Length
sequence_length = 20 # of words in a sequence
# Batch Size
batch_size = 32

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [14]:
# Training parameters
# Number of Epochs
num_epochs = 20
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = len(vocab_to_int)
# Embedding Dimension
embedding_dim = 200
# Hidden Dimension
hidden_dim = 300
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.2)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 20 epoch(s)...
2018-12-15 00:44:42.323508 - Epoch:    1/20    Loss: 5.705016584396362

2018-12-15 00:44:46.890328 - Epoch:    1/20    Loss: 5.129854125976562

2018-12-15 00:44:51.449477 - Epoch:    1/20    Loss: 4.92460218000412

2018-12-15 00:44:56.013525 - Epoch:    1/20    Loss: 4.834421938419342

2018-12-15 00:45:00.583943 - Epoch:    1/20    Loss: 4.675980043888092

2018-12-15 00:45:05.159280 - Epoch:    1/20    Loss: 4.65953959608078

2018-12-15 00:45:09.728600 - Epoch:    1/20    Loss: 4.572981203079224

2018-12-15 00:45:14.321158 - Epoch:    1/20    Loss: 4.591393857479096

2018-12-15 00:45:18.911216 - Epoch:    1/20    Loss: 4.563507309436798

2018-12-15 00:45:23.505691 - Epoch:    1/20    Loss: 4.433701128005981

2018-12-15 00:45:28.073960 - Epoch:    1/20    Loss: 4.438963934898377

2018-12-15 00:45:32.662420 - Epoch:    1/20    Loss: 4.424765236377716

2018-12-15 00:45:37.257144 - Epoch:    1/20    Loss: 4.398823369503021

2018-12-15 00:45:41.857076 - Epoch:   

2018-12-15 00:53:31.430608 - Epoch:    3/20    Loss: 3.8094216461181643

2018-12-15 00:53:36.018028 - Epoch:    3/20    Loss: 3.851935830116272

2018-12-15 00:53:40.611453 - Epoch:    3/20    Loss: 3.8256324772834778

2018-12-15 00:53:45.205261 - Epoch:    3/20    Loss: 3.845305325984955

2018-12-15 00:53:49.792809 - Epoch:    3/20    Loss: 3.795619831085205

2018-12-15 00:53:54.382857 - Epoch:    3/20    Loss: 3.8237466974258423

2018-12-15 00:53:58.954357 - Epoch:    3/20    Loss: 3.8160805315971373

2018-12-15 00:54:03.541287 - Epoch:    3/20    Loss: 3.8421162967681886

2018-12-15 00:54:08.124639 - Epoch:    3/20    Loss: 3.7999069023132326

2018-12-15 00:54:12.710759 - Epoch:    3/20    Loss: 3.8793676862716673

2018-12-15 00:54:17.295541 - Epoch:    3/20    Loss: 3.85525217628479

2018-12-15 00:54:21.876186 - Epoch:    3/20    Loss: 3.8716132698059083

2018-12-15 00:54:26.458695 - Epoch:    3/20    Loss: 3.8494509472846983

2018-12-15 00:54:31.016003 - Epoch:    3/20    Loss: 3.8

2018-12-15 01:02:15.550354 - Epoch:    5/20    Loss: 3.672750936985016

2018-12-15 01:02:20.099701 - Epoch:    5/20    Loss: 3.6639560713768007

2018-12-15 01:02:24.663935 - Epoch:    5/20    Loss: 3.6838734197616576

2018-12-15 01:02:29.242823 - Epoch:    5/20    Loss: 3.6810465841293336

2018-12-15 01:02:33.819658 - Epoch:    5/20    Loss: 3.684172818183899

2018-12-15 01:02:38.378618 - Epoch:    5/20    Loss: 3.659356243610382

2018-12-15 01:02:42.933689 - Epoch:    5/20    Loss: 3.699145807504654

2018-12-15 01:02:47.499277 - Epoch:    5/20    Loss: 3.6875512294769286

2018-12-15 01:02:52.052941 - Epoch:    5/20    Loss: 3.726038139820099

2018-12-15 01:02:56.634765 - Epoch:    5/20    Loss: 3.80265721988678

2018-12-15 01:03:01.218285 - Epoch:    5/20    Loss: 3.750460564136505

2018-12-15 01:03:05.801535 - Epoch:    5/20    Loss: 3.7021905517578126

2018-12-15 01:03:10.383599 - Epoch:    5/20    Loss: 3.7321845951080324

2018-12-15 01:03:14.962040 - Epoch:    5/20    Loss: 3.7458

2018-12-15 01:10:58.192901 - Epoch:    7/20    Loss: 3.5471618432998655

2018-12-15 01:11:02.772038 - Epoch:    7/20    Loss: 3.5918610045909882

2018-12-15 01:11:07.351360 - Epoch:    7/20    Loss: 3.6266940898895266

2018-12-15 01:11:11.927361 - Epoch:    7/20    Loss: 3.6523971400260926

2018-12-15 01:11:16.498610 - Epoch:    7/20    Loss: 3.6295432772636413

2018-12-15 01:11:21.070355 - Epoch:    7/20    Loss: 3.646351330280304

2018-12-15 01:11:25.641439 - Epoch:    7/20    Loss: 3.6862103691101074

2018-12-15 01:11:30.213469 - Epoch:    7/20    Loss: 3.606693299770355

2018-12-15 01:11:34.769158 - Epoch:    7/20    Loss: 3.6476403708457945

2018-12-15 01:11:39.330441 - Epoch:    7/20    Loss: 3.6805604486465455

2018-12-15 01:11:43.914880 - Epoch:    7/20    Loss: 3.6692379727363584

2018-12-15 01:11:48.489080 - Epoch:    7/20    Loss: 3.670944001674652

2018-12-15 01:11:53.048882 - Epoch:    7/20    Loss: 3.671149724960327

2018-12-15 01:11:57.602757 - Epoch:    7/20    Loss: 3.

2018-12-15 01:19:45.151040 - Epoch:    9/20    Loss: 3.5521773672103882

2018-12-15 01:19:49.711943 - Epoch:    9/20    Loss: 3.545239525318146

2018-12-15 01:19:54.270608 - Epoch:    9/20    Loss: 3.5791467480659485

2018-12-15 01:19:58.823494 - Epoch:    9/20    Loss: 3.6122700276374817

2018-12-15 01:20:03.379293 - Epoch:    9/20    Loss: 3.532931074142456

2018-12-15 01:20:07.955194 - Epoch:    9/20    Loss: 3.5797678122520447

2018-12-15 01:20:12.535491 - Epoch:    9/20    Loss: 3.6144015946388244

2018-12-15 01:20:17.100222 - Epoch:    9/20    Loss: 3.594846254348755

2018-12-15 01:20:21.649766 - Epoch:    9/20    Loss: 3.6061499259471894

2018-12-15 01:20:26.197689 - Epoch:    9/20    Loss: 3.6543431763648986

2018-12-15 01:20:30.771153 - Epoch:    9/20    Loss: 3.6204083647727967

2018-12-15 01:20:35.325182 - Epoch:    9/20    Loss: 3.6004328351020813

2018-12-15 01:20:39.875040 - Epoch:    9/20    Loss: 3.6415992436408997

2018-12-15 01:20:44.425751 - Epoch:    9/20    Loss: 3

...

2018-12-15 01:37:10.759105 - Epoch:   13/20    Loss: 3.523825819015503

2018-12-15 01:37:15.310646 - Epoch:   13/20    Loss: 3.5284147698879242

2018-12-15 01:37:19.859086 - Epoch:   13/20    Loss: 3.5350052962303162

2018-12-15 01:37:24.411537 - Epoch:   13/20    Loss: 3.4827543778419496

2018-12-15 01:37:28.966459 - Epoch:   13/20    Loss: 3.5322229676246644

2018-12-15 01:37:33.519927 - Epoch:   13/20    Loss: 3.5140431990623475

2018-12-15 01:37:38.086858 - Epoch:   13/20    Loss: 3.581217432975769

2018-12-15 01:37:42.643970 - Epoch:   13/20    Loss: 3.523340418100357

2018-12-15 01:37:47.211904 - Epoch:   13/20    Loss: 3.535379241466522

2018-12-15 01:37:51.782759 - Epoch:   13/20    Loss: 3.535731608390808

2018-12-15 01:37:56.351157 - Epoch:   13/20    Loss: 3.5028083267211914

2018-12-15 01:38:00.931897 - Epoch:   13/20    Loss: 3.5423400914669037

2018-12-15 01:38:05.506261 - Epoch:   13/20    Loss: 3.523552806377411

2018-12-15 01:38:10.054855 - Epoch:   13/20    Loss: 3.55

2018-12-15 01:45:58.415596 - Epoch:   15/20    Loss: 3.4606656303405763

2018-12-15 01:46:02.973114 - Epoch:   15/20    Loss: 3.4958301413059236

2018-12-15 01:46:07.533174 - Epoch:   15/20    Loss: 3.518261242389679

2018-12-15 01:46:12.107018 - Epoch:   15/20    Loss: 3.5044669008255003

2018-12-15 01:46:16.657625 - Epoch:   15/20    Loss: 3.472392027378082

2018-12-15 01:46:21.205053 - Epoch:   15/20    Loss: 3.542096918106079

2018-12-15 01:46:25.756124 - Epoch:   15/20    Loss: 3.5054414620399474

2018-12-15 01:46:30.303668 - Epoch:   15/20    Loss: 3.4846176562309266

2018-12-15 01:46:34.852066 - Epoch:   15/20    Loss: 3.550351591348648

2018-12-15 01:46:39.412738 - Epoch:   15/20    Loss: 3.5114482274055483

2018-12-15 01:46:43.981253 - Epoch:   15/20    Loss: 3.5141335809230805

2018-12-15 01:46:48.549694 - Epoch:   15/20    Loss: 3.518851667165756

2018-12-15 01:46:53.118803 - Epoch:   15/20    Loss: 3.5056075377464295

2018-12-15 01:46:57.672776 - Epoch:   15/20    Loss: 3.5

2018-12-15 01:54:40.705062 - Epoch:   17/20    Loss: 3.4806033432483674

2018-12-15 01:54:45.261289 - Epoch:   17/20    Loss: 3.4672499442100526

2018-12-15 01:54:49.815154 - Epoch:   17/20    Loss: 3.4885466265678406

2018-12-15 01:54:54.368718 - Epoch:   17/20    Loss: 3.5306787514686584

2018-12-15 01:54:58.928650 - Epoch:   17/20    Loss: 3.4825583505630493

2018-12-15 01:55:03.509758 - Epoch:   17/20    Loss: 3.5143319354057314

2018-12-15 01:55:08.085095 - Epoch:   17/20    Loss: 3.544868454456329

2018-12-15 01:55:12.660997 - Epoch:   17/20    Loss: 3.4799457762241364

2018-12-15 01:55:17.230897 - Epoch:   17/20    Loss: 3.521187197446823

2018-12-15 01:55:21.799512 - Epoch:   17/20    Loss: 3.489853714466095

2018-12-15 01:55:26.369576 - Epoch:   17/20    Loss: 3.4542622747421263

2018-12-15 01:55:30.932090 - Epoch:   17/20    Loss: 3.500740417480469

2018-12-15 01:55:35.481761 - Epoch:   17/20    Loss: 3.493625428676605

2018-12-15 01:55:40.052277 - Epoch:   17/20    Loss: 3.5

2018-12-15 02:03:28.125813 - Epoch:   19/20    Loss: 3.4500212845802305

2018-12-15 02:03:32.707619 - Epoch:   19/20    Loss: 3.5430076823234558

2018-12-15 02:03:37.275212 - Epoch:   19/20    Loss: 3.452685345172882

2018-12-15 02:03:41.861310 - Epoch:   19/20    Loss: 3.5024902830123903

2018-12-15 02:03:46.455646 - Epoch:   19/20    Loss: 3.4790375862121583

2018-12-15 02:03:51.053086 - Epoch:   19/20    Loss: 3.498818504810333

2018-12-15 02:03:55.639003 - Epoch:   19/20    Loss: 3.455312439918518

2018-12-15 02:04:00.205046 - Epoch:   19/20    Loss: 3.524436241388321

2018-12-15 02:04:04.764221 - Epoch:   19/20    Loss: 3.5123690769672393

2018-12-15 02:04:09.320159 - Epoch:   19/20    Loss: 3.490806855201721

2018-12-15 02:04:13.889903 - Epoch:   19/20    Loss: 3.489783372879028

2018-12-15 02:04:18.444638 - Epoch:   19/20    Loss: 3.5367429158687593

2018-12-15 02:04:23.011180 - Epoch:   19/20    Loss: 3.5030623543262482

2018-12-15 02:04:27.592892 - Epoch:   19/20    Loss: 3.49

/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** (Write answer, here)
When increasing the hidden_dim, the time the backpropogation takes drastically increases. 
When increasing the hidden_dim and embedding size, the learn rate can increase and retain a stable loss descent gradient.

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [26]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'elaine' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:49: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


elaine: hoo!(honk) yodel lay hee hoo. i got to get a sample of her, and the fork, they got a lot of pressure.

jerry: oh, i don't know.

helen: morty, are you going to be murphy?

morty: yeah.

morty: oh, i don't believe this.

jerry: oh, you want it?

kramer: i don't understand. it's not fair, but it is the same.

jerry: oh, i know.

elaine:(to george) i don't know.

george: i don't think so.

jerry: i don't know.

elaine: what do you think?

jerry: i know, i was just wondering if it was quite a great idea about it.

george: you know what?

elaine: i don't know.

elaine: what about? you know, i have no idea where everybody's talkin'.

jerry: i don't want it.

elaine: what is it?

kramer: it's a pizza.

george: oh, it's a label maker. i mean, i don't think she'd be able to reach the movie.

kramer: oh, you know, i'm gonna kill him to talk.

jerry: i can't believe this.

elaine: what?

elaine:(still rubbing her nose)

elaine: you mean it was an accident?

elaine: yeah, i think it would 

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [27]:
# save script to a text file
f =  open("generated_script_3.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.